The output is a batch of feature vectors. For each input image, the feature vector has size num_features = 2048. The feature vectors can then be used further, e.g., for classification as above.

The input images are expected to have color values in the range [0,1], following the common image input conventions. The expected size of the input images is height x width = 299 x 299 pixels by default, but other input sizes are possible (within limits).



In [19]:
%reset -f

In [22]:
dataset_path = "C:/Users/vaibhav.jadhav/Downloads/seed_dataset"

In [23]:
# Transfer learning model used is Inception V3, which is trained on 229x229 dimentional input images.
# spectrograms created from EEG signal will be fed as input to "inception v3 with top classification layer removed"
# this will give us feature vector of size 2048 as inception v3's layer before classification layer outputs 2048 dim vector.
spectrogram_height = 229
spectrogram_width = 229
hop_size = 3

frame_size = ((spectrogram_height-1) * 2)
sample_size = ((spectrogram_width-1) * hop_size)

In [24]:
# sampling freq of EEG, provided by data owner
fs = 200

In [25]:
import os
import sys
import math
import numpy as np
# import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from scipy.io import loadmat

In [26]:
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt

In [27]:
import tensorflow_hub as hub

In [28]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [29]:
def compute_spectrogram(signal, sr=fs,frame_size = frame_size, hop = hop_size):
    stft = librosa.stft(signal, n_fft=frame_size, hop_length=hop)
    log_stft= librosa.power_to_db(np.abs(stft) ** 2)
    return log_stft
#     print(log_stft.shape)
#     plot_spectrogram(log_stft, sr, hop, y_axis="log")

In [30]:
def plot_spectrogram(Y, sr=fs, hop_length=hop_size, y_axis="linear"):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y, 
                             sr=sr, 
                             hop_length=hop_length, 
                             x_axis="time", 
                             y_axis=y_axis)
    plt.colorbar(format="%+2.f")

In [31]:
fs = 200

def decompose_ram_efficient(file, file_mat, exp_id):
    # trial*channel*sample
    feature_extractor_model = "https://tfhub.dev/google/imagenet/inception_v3/feature_vector/5"
    out_dim_of_TL_model = 2048
    data = loadmat(file)
    frequency = fs
#     time_window = 0.5
#     data_points_per_sample = frequency * time_window  #fz = 200per sec -> 100 data_points per 0.5sec window
    data_points_per_sample = sample_size
    name = ""
    for char in list(data.keys())[7]: #instead of 7, u can pass any no. between 3 and 15
      if char == "_":
        break
      name = name+char

#     fft_crop_length = 21 

    decomposed_specgram = np.empty([0, 62, 229, 229, 3])
    label = np.array([])
    all_label = [1, 0, -1, -1, 0, 1, -1, 0, 1, 1, 0, -1, 0, 1, -1]

    x_TL_features = np.empty([0, 62, out_dim_of_TL_model])
    
    
    for trial in range(15):
        tmp_trial_signal = data[name + '_eeg' + str(trial + 1)]
        num_sample = int(len(tmp_trial_signal[0]) / data_points_per_sample) 
        print('{}-{}'.format(trial + 1, num_sample))

        # temp_fft = np.empty([0, num_sample, 21])
        temp_specgram = np.empty([0, 229, 229])
        label = np.append(label, [all_label[trial]] * num_sample)

        for channel in range(62):
            trial_signal = tmp_trial_signal[channel]

            delta_theta_alpha = butter_bandpass_filter(trial_signal, 1, 14, frequency, order=3)
            delta_theta_alpha = delta_theta_alpha+abs(np.min(delta_theta_alpha))
            delta_theta_alpha = delta_theta_alpha/np.max(delta_theta_alpha)
            
            beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
            beta = beta+abs(np.min(beta))
            beta = beta/np.max(beta)
            
            gamma = butter_bandpass_filter(trial_signal, 31, 51, frequency, order=3)
            gamma = gamma+abs(np.min(gamma))
            gamma = gamma/np.max(gamma)

            specgram_delta_theta_alpha = np.zeros(shape=[0, 229, 229], dtype=float)
            specgram_beta = np.zeros(shape=[0, 229, 229], dtype=float)
            specgram_gamma = np.zeros(shape=[0, 229, 229], dtype=float)

            for index in range(num_sample):
                specgram_delta_theta_alpha = np.vstack([specgram_delta_theta_alpha, compute_spectrogram(delta_theta_alpha[index * sample_size:(index + 1) * sample_size]).reshape(1,229,229)])
                specgram_beta = np.vstack([specgram_beta, compute_spectrogram(beta[index * sample_size:(index + 1) * sample_size]).reshape(1,229,229)])
                specgram_gamma = np.vstack([specgram_gamma, compute_spectrogram(gamma[index * sample_size:(index + 1) * sample_size]).reshape(1,229,229)])
                # print(fft_delta.shape, fft_theta.shape, fft_alpha.shape, fft_beta.shape, fft_gamma.shape)

            temp_specgram = np.vstack([temp_specgram, specgram_delta_theta_alpha])
            temp_specgram = np.vstack([temp_specgram, specgram_beta])
            temp_specgram = np.vstack([temp_specgram, specgram_gamma])

        temp_trial_specgram = temp_specgram.reshape(-1, 3, num_sample, 229, 229)
        temp_trial_specgram = temp_trial_specgram.transpose([2, 0, 3, 4, 1])
        subject_id = ""
        for i in file_mat:
            if i == "_":
                break
            subject_id += i
#         np.save(f"{dataset_path}/my_extracted_features/spectrogram(229_229_3)/{subject_id}_{trial+1}_X_1D.npy",temp_trial_specgram)
#         decomposed_specgram = np.vstack([decomposed_specgram, temp_trial_specgram])
#         
        dataset_path = "C:/Users/vaibhav.jadhav/Downloads/seed_dataset"

        subject = subject_id
#         trial_group = en+1


        temp_x = temp_trial_specgram #(#,62,229,229,3)

        print(f"subject:{subject}, exp:{exp_id} and trial:{trial+1} has specgrams as: {temp_x.shape}")

        # spectrogram creation complete
        
############################# TL start ##################################    
        # starting Transefer learning, forward pass in inception v3 with top layer removed
        pretrained_model_without_top_layer = hub.KerasLayer(
            feature_extractor_model, input_shape=(229, 229, 3), trainable=False)

        
        for sample in range(temp_x.shape[0]):
            temp_x_TL_features = np.empty([0, out_dim_of_TL_model])
            for channel in range(temp_x.shape[1]):
                results = ((np.array(pretrained_model_without_top_layer(temp_x[sample,channel].reshape((1,229,229,3))))))
                temp_x_TL_features = np.vstack([temp_x_TL_features, results])
            temp_x_TL_features = temp_x_TL_features.reshape((1,temp_x_TL_features.shape[0],out_dim_of_TL_model))
            x_TL_features = np.vstack([x_TL_features, temp_x_TL_features])
            
        temp_x = None 
        temp_trial_specgram = None

############################# TL done ##################################        
    
####converting EEG signals to 8x9 feature map, to capture their spatial information and pass to CNN model later ####
####for more information about 8x9 map, refer to paper with following link or DOI ####
# link: https://www.itm-conferences.org/articles/itmconf/abs/2023/03/itmconf_icdsia2023_02011/itmconf_icdsia2023_02011.html
# DOI: https://doi.org/10.1051/itmconf/20235302011
    print(f"TL_features_for subject:{subject} and exp:{exp_id} -> {x_TL_features.shape}")

    X89 = np.zeros((x_TL_features.shape[0], 8, 9, out_dim_of_TL_model))
    
    X89[:, 0, 2, :] = x_TL_features[:, 3, :]
    X89[:, 0, 3:6, :] = x_TL_features[:, 0:3, :]
    X89[:, 0, 6, :] = x_TL_features[:, 4, :]

    for i in range(5):
      X89[:, i + 1, :, :] = x_TL_features[:, 5 + i * 9:5 + (i + 1) * 9, :]

    X89[:, 6, 1:8, :] = x_TL_features[:, 50:57, :]
    X89[:, 7, 2:7, :] = x_TL_features[:, 57:62, :]
    
    print(f"8*9 map dim for subject:{subject} and exp:{exp_id} is -> {X89.shape}")


    np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_X89.npy",X89)
    np.save(f"{dataset_path}/my_extracted_features/spectrogram(224_224_3)/TL_inception_v3/{subject}_{exp_id}_y.npy",label)

    print("######################################################")


#         %reset -f
    

In [33]:
########### extracting features from seed data ################

In [36]:
file_path = f"{dataset_path}/Preprocessed_EEG/"


In [37]:

files = os.listdir(file_path)
files.sort(key= (lambda i : i[:2] if i[1]!="_" else "0"+i[0]) )
print(files)

['1_20131027.mat', '1_20131030.mat', '1_20131107.mat', '2_20140404.mat', '2_20140413.mat', '2_20140419.mat', '3_20140603.mat', '3_20140611.mat', '3_20140629.mat', '4_20140621.mat', '4_20140702.mat', '4_20140705.mat', '5_20140411.mat', '5_20140418.mat', '5_20140506.mat', '6_20130712.mat', '6_20131016.mat', '6_20131113.mat', '7_20131027.mat', '7_20131030.mat', '7_20131106.mat', '8_20140511.mat', '8_20140514.mat', '8_20140521.mat', '9_20140620.mat', '9_20140627.mat', '9_20140704.mat', '10_20131130.mat', '10_20131204.mat', '10_20131211.mat', '11_20140618.mat', '11_20140625.mat', '11_20140630.mat', '12_20131127.mat', '12_20131201.mat', '12_20131207.mat', '13_20140527.mat', '13_20140603.mat', '13_20140610.mat', '14_20140601.mat', '14_20140615.mat', '14_20140627.mat', '15_20130709.mat', '15_20131016.mat', '15_20131105.mat', 'label.mat', 'readme.txt']


In [38]:
print(files.pop())
print(files.pop())
len(files)

readme.txt
label.mat


45

In [18]:
files[8],files[9],files[10]

('3_20140629.mat', '4_20140621.mat', '4_20140702.mat')

In [39]:
# X = np.empty([0, 62, 224, 224, 3])
# y = np.empty([0, 1])

exp_id = 0
for i in files[3:]:
    exp_id+=1
    file_name = file_path + i
    print(f'processing {i}')
    decompose_ram_efficient(file_name, i, exp_id)
    if exp_id == 3:
        exp_id = 0
    

processing 2_20140404.mat
1-68
subject:2, exp:1 and trial:1 has specgrams as: (68, 62, 229, 229, 3)
2-68
subject:2, exp:1 and trial:2 has specgrams as: (68, 62, 229, 229, 3)
3-60
subject:2, exp:1 and trial:3 has specgrams as: (60, 62, 229, 229, 3)
4-69
subject:2, exp:1 and trial:4 has specgrams as: (69, 62, 229, 229, 3)
5-54
subject:2, exp:1 and trial:5 has specgrams as: (54, 62, 229, 229, 3)
6-57
subject:2, exp:1 and trial:6 has specgrams as: (57, 62, 229, 229, 3)
7-69
subject:2, exp:1 and trial:7 has specgrams as: (69, 62, 229, 229, 3)
8-63
subject:2, exp:1 and trial:8 has specgrams as: (63, 62, 229, 229, 3)
9-77
subject:2, exp:1 and trial:9 has specgrams as: (77, 62, 229, 229, 3)
10-69
subject:2, exp:1 and trial:10 has specgrams as: (69, 62, 229, 229, 3)
11-68
subject:2, exp:1 and trial:11 has specgrams as: (68, 62, 229, 229, 3)
12-68
subject:2, exp:1 and trial:12 has specgrams as: (68, 62, 229, 229, 3)
13-68
subject:2, exp:1 and trial:13 has specgrams as: (68, 62, 229, 229, 3)
14-6

5-54
subject:4, exp:1 and trial:5 has specgrams as: (54, 62, 229, 229, 3)
6-57
subject:4, exp:1 and trial:6 has specgrams as: (57, 62, 229, 229, 3)
7-69
subject:4, exp:1 and trial:7 has specgrams as: (69, 62, 229, 229, 3)
8-63
subject:4, exp:1 and trial:8 has specgrams as: (63, 62, 229, 229, 3)
9-77
subject:4, exp:1 and trial:9 has specgrams as: (77, 62, 229, 229, 3)
10-69
subject:4, exp:1 and trial:10 has specgrams as: (69, 62, 229, 229, 3)
11-68
subject:4, exp:1 and trial:11 has specgrams as: (68, 62, 229, 229, 3)
12-68
subject:4, exp:1 and trial:12 has specgrams as: (68, 62, 229, 229, 3)
13-68
subject:4, exp:1 and trial:13 has specgrams as: (68, 62, 229, 229, 3)
14-69
subject:4, exp:1 and trial:14 has specgrams as: (69, 62, 229, 229, 3)
15-60
subject:4, exp:1 and trial:15 has specgrams as: (60, 62, 229, 229, 3)
TL_features_for subject:4 and exp:1 -> (987, 62, 2048)
8*9 map dim for subject:4 and exp:1 is -> (987, 8, 9, 2048)
######################################################
proc

KeyboardInterrupt: 

In [ ]:
trail_samples = {1:68,2:68,3:60,4:69,5:54,6:57,7:69,8:63,9:77,10:69,11:68,12:68,13:68,14:69,15:60}

In [21]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
